In [ ]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import imageio as io
import os

#ficheiros .py acessórios
import automatic_patching as patcher
import utils
import generate_data as generator
import evaluator as evaluator

# Step-by-step

In [ ]:
patient_number = 15

In [ ]:
patient_folder = "patients/patient_{}/".format(patient_number)

In [ ]:
le = LabelEncoder()
ground_truth_csv = pd.read_csv('patients/patient_gt.csv')
df_list = [df for df in ground_truth_csv.groupby('patient')]
for df in df_list:
    if 'patient_{}'.format(patient_number) == df[0]:
        gt = list(df[1]['label'])[0]
        patient_df = df[1]
        patient_df = patient_df.reset_index()
        density = list(df[1]['density'])[0]

### Show mammograms

In [ ]:
print('Patient {} | {} | Density: {}'.format(patient_number,gt,density))
patcher.show_mammograms(patient_folder)

### Show binary mammogram

In [ ]:
patcher.show_binarized(patient_folder)

In [ ]:
    threshold = threshold_otsu(raw_mammogram_array)
    #threshold = threshold_yen(raw_mammogram_array)
    binarized_array = (raw_mammogram_array > threshold).astype(np.int_)
    binarized_filled= scipy.ndimage.binary_fill_holes(binarized_array).astype(int)

### Show number of patches

In [ ]:
patch_size = 300
overlap = 0.2

In [ ]:
patcher.patches_number(patient_folder,patch_size,overlap)

### Show patch sides and centers

In [ ]:
patcher.show_sides(patient_folder,patch_size,overlap)

In [ ]:
patcher.show_centers(patient_folder,patch_size,overlap)

## Classification

### without Data Augmentation

In [ ]:
try:
    loaded_model = keras.models.load_model("D:/Lesion-Based Patches/vgg19_trained",compile=True)
except OSError:
    print("Failed to load DL model!") 
else:
    print("Successfully loaded DL model!")

### Classify patches by image

In [ ]:
predictions_d = dict()
image_paths = os.listdir(patient_folder)
not_images = []
for image in image_paths:
    if 't' in image:
        not_images.append(image)
for image in not_images:
    image_paths.remove(image)
for image in image_paths:
    path = patient_folder+image
    patches_paths = patcher.save_patches_by_image(path,patch_size,overlap)
    df = pd.DataFrame({'paths':patches_paths,'labels':'Test'})
    generated_data = generator.generate(df,patch_size)
    predictions = loaded_model.predict(generated_data,verbose=1)
    print("Successfully classified patches!")
    print('\n')
    predictions_image=[]
    for i in range(len(predictions)):
        predictions_image.append(predictions[i][0])
    predictions_d[image] = predictions_image

In [ ]:
print('Patient {} | {} | Density: {}'.format(patient_number,gt,density))
for image_view,i_classifications in predictions_d.items():
    image_view = image_view.split('.')[0]
    ratios = []
    num_patches = len(i_classifications)
    positive_patches = 0
    for pred in i_classifications:
        if pred > 0.95:
            positive_patches += 1
    ratio = positive_patches/num_patches
    ratios.append(ratio)
    print(image_view +': Patches {} | Positive Patches {} | Ratio {:.2f}'.format(num_patches,positive_patches,ratio))
#print('Average Ratio: ',round(average_ratio(ratios),2))

### Clasify patches by case

In [ ]:
patches_paths = patcher.save_patches_by_case(patient_folder,patch_size,overlap)

In [ ]:
df = pd.DataFrame({'paths':patches_paths,'labels':'Test'})
generated_data = generator.generate(df,patch_size)

In [ ]:
predictions = loaded_model.predict(generated_data,verbose=1)
print("Successfully classified patches!")
predictions_case=[]
for i in range(len(predictions)):
    predictions_case.append(predictions[i][0])

In [ ]:
print('Patient {} | {} | Density: {}'.format(patient_number,gt,density))
num_patches = len(predictions_case)
positive_patches = 0
for pred in predictions_case:
    if pred > 0.95:
        positive_patches += 1
ratio = positive_patches/num_patches
print('Patches {} | Positive Patches {} | Ratio {:.3f}'.format(num_patches,positive_patches,ratio))

In [ ]:
evaluator.make_hist(predictions_case,patient_number,gt)

In [ ]:
patcher.show_positive_patches_on_mammography(patient_folder,patch_size,overlap,predictions_case,patient_df)

In [ ]:
0/0

# 1º - Verification Step

# 2º - Patching

# 3º - Classification

# Patching e Classificação

In [ ]:
patient_folder = "patients/"
patient_paths = os.listdir(patient_folder)
not_images = []
for image in patient_paths:
    if 'g' in image:
        not_images.append(image)
for image in not_images:
    patient_paths.remove(image)
patient_paths = utils.sort_paths(patient_paths)

In [ ]:
try:
    loaded_model = keras.models.load_model("D:/Lesion-Based Patches/vgg19_trained",compile=True)
    print("Successfully loaded DL model!")
except ValueError:
    print("Failed to load DL model!") 

In [ ]:
patch_size = 300
overlap = 0.2

## Por imagem

In [ ]:
all_classifications = dict()
for patient in patient_paths:    
    case_classification = dict()
    images = os.listdir(patient_folder+patient)
    not_images = []
    for image in images:
        if 't' in image:
            not_images.append(image)
    for image in not_images:
        images.remove(image)    
    for image_view in images:
        image_type = image_view.split(".")[0]
        path = patient_folder+patient+"/"+image_view
        saved_patches_paths = patcher.save_patches_by_image(path,patch_size,overlap)
        df = pd.DataFrame({'paths':saved_patches_paths,'labels':'Test'})
        generated_data = generator.generate(df,input_size=300)
        predictions = loaded_model.predict(generated_data)
        predictions_image=[]
        for i in range(len(predictions)):
            predictions_image.append(predictions[i][0])
        case_classification[image_type] = predictions_image
    print("Successfully classified images from {}!".format(patient))
    print('\n')
    all_classifications[patient] = case_classification
le = LabelEncoder()
ground_truth_csv = pd.read_csv('patients/patient_gt.csv')
evaluator.report_by_image(all_classifications,ground_truth_csv)

## Por caso

In [ ]:
all_classifications = dict()
for patient in patient_paths:
    path = 
    saved_patches_paths = patcher.save_patches_by_case(,patch_size,overlap)
    df = pd.DataFrame({'paths':saved_patches_paths,'labels':'Test'})
    generated_data = generator.generate(df,input_size=300)
    predictions = loaded_model.predict(generated_data)
    predictions_case=[]
    for i in range(len(predictions)):
        predictions_case.append(predictions[i][0])
    all_classification[patient] = predictions_case
    print("Successfully classified patches from {}!".format(patient))
    print('\n')
le = LabelEncoder()
ground_truth_csv = pd.read_csv('patients/patient_gt.csv')
ground_truth = le.fit_transform(list(ground_truth_csv['label']))
evaluator.report_by_case(all_classifications,ground_truth_csv)

In [ ]:
df_list = [df for df in ground_truth_csv.groupby('patient')]
for df in df_list:
    print(df[0])

In [ ]:
pd.read_csv('D:/Architecture/classification_report_by_case.csv')

In [ ]:
a = pd.read_csv('D:/Architecture/classification_report_by_image.csv')
a.head(30)

# Apenas Classificação

In [ ]:
patient_folder_path = "patients/"
image_paths = utils.sort_paths(os.listdir(patient_folder_path))

In [ ]:
try:
    loaded_model = keras.models.load_model("D:/BCDR/vgg19_trained",compile=True)
    print("Successfully loaded DL model!")
except OSError:
    print("Failed to load DL model!") 

In [ ]:
all_classifications = dict()
for patient in image_paths:    
    case_classification = dict()
    images = os.listdir(patient_folder_path+patient)
    not_images = []
    for image in images:
        if 't' in image:
            not_images.append(image)
    for image in not_images:
        images.remove(image)
    for image_view in images:
        image_type = image_view.split(".")[0]
        path = patient_folder_path+patient+"/"+image_view
        saved_patches_paths = patching_tool.only_paths(path)
        df = pd.DataFrame({'paths':saved_patches_paths,'labels':'Test'})
        generated_data = generator_tool.generate(df)
        with tf.device('/gpu'):
            predictions = loaded_model.predict(generated_data)
        predictions_image=[]
        for i in range(len(predictions)):
            predictions_image.append(predictions[i][0])
        case_classification[image_type] = predictions_image
    print("Successfully classified images from {}!".format(patient))
    print('\n')
    all_classifications[patient] = case_classification

In [ ]:
le = LabelEncoder()
ground_truth_csv = pd.read_csv('patients/patient_gt.csv')
ground_truth = le.fit_transform(list(ground_truth_csv['label']))
densities = list(ground_truth_csv['density'])

In [ ]:
evaluator.report(all_classifications,ground_truth,densities)

## Single Threshold Classification

In [ ]:
predictions_single = evaluator.calculate_threshold_predictions(all_classifications,0.5)

In [ ]:
df_single,auc = evaluator.evaluate_predictions(image_paths,ground_truth,predictions_single)
print('AUC :',round(auc,4))

## Double Threshold Classification

In [ ]:
predictions_double,g_truth,patient_list = evaluator.calculate_case_predictions(all_classifications,list(ground_truth),image_paths,0.3,0.7)

In [ ]:
df_double,auc = evaluator.evaluate_predictions(patient_list,g_truth,predictions_double)
print('AUC :',round(auc,4))